In [1]:
library(rwwa)

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: plyr


Attaching package: ‘plyr’


The following object is masked from ‘package:distillery’:

    is.formula




## Per grid cell

In [2]:
gmst <- read.table("gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2023]

nino <- read.csv("nino34_djf.csv", col.names = c("year", "nino"))

cov_2023 <- data.frame(gmst = gmst$gmst[gmst$year == 2023], nino = nino$nino[nino$year == 2023])
cov_cf <- rbind("hist" = cov_2023 - c(1.2, 0),
                "neut" = c(cov_2023$gmst, 0))

In [ ]:
fl = list.files(pattern = "tamsat_pr-djf_flattened")

for(fnm in fl) {
    df <- read.csv(fnm, skip = 3, header = F)
    colnames(df)[1] <- "year"
    df <- merge(merge(nino, gmst), df)
    
    # fit nonstationary model to all grid cells
    models <- sapply(colnames(df)[-(1:3)], function(cnm) {
        df_c <- df[,c("year", "gmst", "nino", cnm)]
        fit_ns("norm", "fixeddisp", df_c, cnm, c("gmst", "nino"), lower = T)
    }, simplify = F)
    
    # get all model results & save as .csv to be reconstructed & plotted in python
    fitted_res <- sapply(models, mdl_ests, cov_f = cov_2023, cov_cf = cov_cf)
    fitted_res <- rbind(fitted_res, rp_neut = sapply(models, function(mdl) return_period(mdl, x = mdl$ev, fixed_cov = cov_cf["neut",,drop=F])))
    write.csv(fitted_res, gsub("flattened", "fitted", fnm))
}